# 训练模型
## 1. 单模型训练  
单模型训练, 我们采用的是keras库提供的InceptionV3, ResNet50, InceptionResNetV2三个模型

数据集下载：https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data

### 载入必要的库

In [6]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.callbacks import *
from keras.applications.imagenet_utils import preprocess_input

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import h5py
import numpy as np
from tqdm import tqdm
import cv2
import math
import os
import datetime
import pydot
from matplotlib import pyplot as plt

### 定义检查路径函数

In [ ]:
def check_path_gen():
    if not(os.path.exists('./Weights')):
        os.mkdir('Weights')
    if not(os.path.exists('./Tensor_log')):
        os.mkdir('Tensor_log')

### 定义训练函数

In [ ]:
def train_cat_dog(MODEL, image_size):
    # hyper-parameters
    bs = 16

    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)

    # 搭建tranfer learning的最后一层
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.25)(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    model.compile(optimizer='adadelta',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # 获取训练数据
    np.random.seed(2017)

    n = 25000
    X = np.zeros((n, image_size[0], image_size[1], 3), dtype=np.uint8)
    y = np.zeros((n, 1), dtype=np.uint8)
    X_file = 'X' + str(image_size[0]) + '.npy'
    y_file = 'y' + str(image_size[0]) + '.npy'

    if os.path.exists(X_file) and os.path.exists(y_file):
        X = np.load(X_file)
        y = np.load(y_file)
    else:
        for i in tqdm(range(int(n/2))):
            X[i] = cv2.resize(cv2.imread('//Dataset/train/cat/cat.%d.jpg' % i), (image_size[0], image_size[1]))
            X[i+int(n/2)] = cv2.resize(cv2.imread('/Dataset/train/dog/dog.%d.jpg' % i), (image_size[0], image_size[1]))
        y[int(n/2):] = 1
        np.save(X_file, X)
        np.save(y_file, y)

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1)

    train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    train_generator = train_datagen.flow(X_train, y_train, batch_size=bs)
    val_generator = val_datagen.flow(X_valid, y_valid, batch_size=bs)

    # callbacks
    best_weights_path = os.path.join('./Weights', MODEL.__name__)
    if not(os.path.exists(best_weights_path)):
        os.mkdir(best_weights_path)
    best_weights_filepath = os.path.join(best_weights_path, 'best_weights')

    log_path = os.path.join('./Tensor_log', MODEL.__name__)
    if not (os.path.exists(log_path)):
        os.mkdir(log_path)

    earlyStopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1, mode='auto')
    saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=1/math.e, verbose=1, patience=10, min_lr=0.0001)
    tensorboard = TensorBoard(log_dir=log_path)

    # 训练
    model.fit_generator(train_generator, steps_per_epoch=1000, epochs=10, validation_data=val_generator,
                        verbose=2, callbacks=[earlyStopping, saveBestModel, reduce_lr, tensorboard])W
    model.save(MODEL.__name__ + '_cvd.h5')

### 执行训练过程, 统计训练时间并生成固化模型

In [ ]:
check_path_gen()

# tick tock start
starttime = datetime.datetime.now()
train_cat_dog(InceptionV3, (299, 299))
endtimeV3 = datetime.datetime.now()

train_cat_dog(ResNet50, (224, 224))
endtimeRes = datetime.datetime.now()

train_cat_dog(InceptionResNetV2, (299, 299))
endtimeIRV2 = datetime.datetime.now()

print('InceptionV3 train time %d seconds' % (endtimeV3-starttime).seconds)
print('ResNet50 train time %d seconds' % (endtimeRes-endtimeV3).seconds)
print('InceptionResNetV2 train time %d seconds' % (endtimeIRV2-endtimeRes).seconds)

## 2. 模型融合训练  
将三个模型特征提取出来, 融合在一起, 在加FC层和分类器, 利用集成学习方法训练更强大的模型

### 定义函数将三个模型的特征分别提取出来并保存成.h5文件

In [ ]:
def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)

    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("./Dataset/train", image_size, shuffle=False,
                                              batch_size=100)
    test_generator = gen.flow_from_directory("./Dataset/test2", image_size, shuffle=False,
                                             batch_size=100, class_mode=None)

    train = model.predict_generator(train_generator, int(train_generator.samples/100))
    test = model.predict_generator(test_generator, int(test_generator.samples/100))
    with h5py.File("gap_%s.h5" % MODEL.__name__) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

### 提取模型特征的执行过程

In [ ]:
starttime = datetime.datetime.now()
write_gap(ResNet50, (224, 224))
write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input)
write_gap(InceptionResNetV2, (299, 299), inception_resnet_v2.preprocess_input)
endtime = datetime.datetime.now()
print((endtime-starttime).seconds)

### 训练融合模型并固化模型

In [ ]:
X_train = []
X_test = []

starttime = datetime.datetime.now()

for filename in ["gap_ResNet50.h5", "gap_InceptionResNetV2.h5", "gap_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

# 训练
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

log_path = './model_concat_tensor_log'
tensorboard = TensorBoard(log_dir=log_path)


model.fit(X_train, y_train, batch_size=128, nb_epoch=8, validation_split=0.2, callbacks=[tensorboard])
endtime = datetime.datetime.now()
print("model concat train time %d seconds" % (endtime-starttime).seconds)

# 模型保存
model.save('model_concat_cvd.h5')

### 模型可视化
我们利用graphviz写好模型的结构, 观察输入和输出的特征数量和经过的结点

In [ ]:
%matplotlib inline

(graph, ) = pydot.graph_from_dot_file('./graph_model_concat.dot')
graph.write('./graph_model_concat.png', format='png')
graph_img = cv2.imread('graph_model_concat.png')
b, g, r = cv2.split(graph_img)
img = cv2.merge([r, g, b])
plt.imshow(graph_img)